In [1]:
# import required classes
import numpy as np
import os
import threading
    
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session

# configure memory allocate for tensorflow backend
config = tf.ConfigProto()
#dynamically grow the memory used on the GPU
config.gpu_options.allow_growth = True  
set_session(tf.Session(config=config))


Using TensorFlow backend.


# Start autoDrone

In [2]:
from drone import Drone
import sys
if "E:\\git\\AirSim\\PythonClient" not in sys.path:
    sys.path.append("E:\\git\\AirSim\\PythonClient")
    print(sys.path)
import airsim

client = airsim.MultirotorClient()

auto_drone = Drone (client)

auto_drone.connect()
auto_drone.takeoff()

# default is 45 degree down
auto_drone.change_camera_pitch(0)

from environment import Environment
env = Environment (client)

['E:\\git\\models\\research', 'E:\\git\\models\\research\\slim', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\python36.zip', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\DLLs', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36', '', 'C:\\Users\\hoang\\AppData\\Roaming\\Python\\Python36\\site-packages', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib\\site-packages', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\hoang\\.ipython', 'E:\\git\\AirSim\\PythonClient']
['E:\\git\\models\\research', 'E:\\git\\models\\research\\slim', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\python36.zip', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\DLLs', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36\\lib', 'C:\\Users\\hoang\\AppData\\Local\\conda\\conda\\envs\\tf36', '

### Load Models (Navigation + Detection)

In [ ]:
# model_id: 0 - Resnet18, 1 - MobileNetV2, 2 - DenseNet121, 3 - NasNetMobile
model = auto_drone.load_navigation_model(1)
# auto_drone.load_detection_graph()

### Sanity test for the model  

In [ ]:
from skimage.io import imread, imsave
import log_handler

f = 'E:\\UAV_drone\\data\\2019-01-04-6groups_120fov_height_normal\\val\\medium\\img_0_0_1546559335497177700.png'
# f = 'img_0_0_1546722126943805600.png'
img = imread(f) 
img_rgb = img[:,:,:3]
img_norm = img_rgb / 255.0
img_final = np.reshape(img_norm,[1,512,512,3])
probs = model.predict(img_final)
print (probs)
offset_prob = probs[0][0]
view_prob = probs[1][0]
height_prob = probs[2][0]
pitch_prob = probs[3][0]
print(pitch_prob)
print(height_prob)
print(offset_prob)
print(view_prob)
img_path = os.path.join('.' , 'prob_'+'.png')
log_handler.write_probs_on_image(pitch_prob, height_prob, offset_prob, view_prob, 0, img_rgb, img_path)

In [ ]:
# import common_functions as control
# from common_types import *
# import airsim


import pprint
import time
import math

def toDegree(rad):
    return rad*180.0/math.pi

def toRad(angle):
    return angle*math.pi/180.0


#### initialize some variables 

In [ ]:
FLYING_LOG_DIR = "E:\\UAV_drone\\Logs\\flying_recording_play_asdd"
if not os.path.exists(FLYING_LOG_DIR):
    os.makedirs(FLYING_LOG_DIR)

name = os.path.join(FLYING_LOG_DIR, "logs.txt") 
f = open(name, "a+")

randomWeatherList =   [[1.0, 1.0, 0.25, 0.0, 0.25],
                       [0.0, 0.0, 0.75, 0.0, 0.0],
                       [0.0, 0.0, 0.25, 0.75, 0.25],
                       [0.0, 0.0, 0.0, 0.0, 0.75],
                       [0.5, 0.5, 0.5, 0.5, 0.5]]

recording = True
change_pitch_count = 0
total_z_angle_prob = 0
user_input = []
adjust_camera_enable = True
changing_weather_enabled = False

#spawn a new thread to wait for input
def get_user_input(user_input): 
    input("Press enter to stop flying: ")
    user_input.append(None)
    


### Main loop to fly drone autonomously

In [ ]:
import log_handler

mythread = threading.Thread(target=get_user_input, args=(user_input,)) 
mythread.daemon = True 
mythread.start()

i = 0 
while True: 
    if (changing_weather_enabled and i%50==0):
        (r, s, fo, l, d) = randomWeatherList[int(i/50)%4]
        env.set_weather(r, s, fo, l, d)

    # get current state of the drone
    _, _, cur_yaw_rad = auto_drone.getPitchRollYawInRad()
    if (recording):
        cur_yaw_degree = toDegree(cur_yaw_rad)
        cur_z = auto_drone.getZ()
        f.write('{}, cur_yaw {:.3f}, cur_z {:.3f}\n'.format(i, cur_yaw_degree, cur_z))

    # observe from the front camera (id=0)
    img3d_rbg = auto_drone.observe(0)
    pitch_prob, height_prob, offset_prob, view_prob = auto_drone.estimate_position_and_orientation(img3d_rbg)
    
    # detect objects
    mast_detections = auto_drone.detect_objects(img3d_rbg, detection_graph, 4)
    
    if (recording): 
        img_path = os.path.join(FLYING_LOG_DIR , 'prob_'+str(i)+'.png')
        log_handler.write_probs_on_image(pitch_prob, height_prob, offset_prob, view_prob, -auto_drone.camera_pitch, img3d_rbg, img_path)

    
    z_angle_prob = pitch_prob[0] - pitch_prob[2] # down - up 

    # check and adjust the camera
    if adjust_camera_enable == True:
        # only change the camera when it is greater then 0
        if (abs(z_angle_prob)>0.5):        
            change_pitch_count += 1
            total_z_angle_prob += z_angle_prob
            if (recording):
                f.write('{}, change_pitch_count {},  total_z_angle_prob {}, vertical_angle is {:.3f}\n'.format(i,change_pitch_count, total_z_angle_prob, auto_drone.camera_pitch))

            if (change_pitch_count >= 3):
                degree_to_change = -(1.0*total_z_angle_prob/change_pitch_count)*10
                auto_drone.change_camera_pitch(degree_to_change)              
                # reset if 3 consecutive times already
                change_pitch_count = 0
                total_z_angle_prob = 0
        else:
            change_pitch_count = 0
            total_z_angle_prob = 0

            
    # calculate height to change
    z_offset = 0.4*(height_prob[0] - height_prob[2]) # low - high
    # double move higher or lower
    if ((height_prob[0] > 0.95 or height_prob[2] >0.95) and (adjust_camera_enable == False or abs(auto_drone.camera_pitch) >= 25.0)):        
        z_offset *= 1.5

    # calculate yaw
    latoff_angle = 10 * (offset_prob[0] - offset_prob[2]) # left - right
    view_angle = 10 * (view_prob[0] - view_prob[2])
    yaw_degree_to_change =  view_angle + latoff_angle 
    
    headless = False 
    if (view_prob[1] >= 0.99):
        headless = True       

    if (recording):
        f.write('{}, added_yaw {:.3f}, added_z {:.3f}, z_angle_prob {:.3f}\n'.format(i, yaw_degree_to_change, z_offset, z_angle_prob))
        f.write('{}, view_angle {:.3f}, latoff_angle {:.3f}\n'.format(i, view_angle, latoff_angle))


    if (headless == False and (yaw_degree_to_change > 10.5 or yaw_degree_to_change < -10.5)):
        if (recording):
            f.write('****turning with a BIG ANGLE\n')
        #double rotate in this case
        auto_drone.rotate(yaw_degree_to_change)

    if adjust_camera_enable == True:
        velocity = 0.25*pitch_prob[1] + 0.25*offset_prob[1] + 0.25*view_prob[1] + 0.25*height_prob[1]
    else:
        velocity = 0.333*offset_prob[1] + 0.333*view_prob[1] + 0.333*height_prob[1]
        
    yaw_rad_to_change = toRad(yaw_degree_to_change)

    auto_drone.moveByYawZ(yaw_rad_to_change, -z_offset, velocity, headless, file=f, recording=True)

    i+=1
    if user_input:
        break 
        
print ("stopped at ", i)
auto_drone.hover()
f.close()

### Intervention code

In [ ]:
f = open(name, "a+")
f.write('intervention, moving drone back to proper position')

In [ ]:
# shilf right
auto_drone.shilf_horizontally('left')

In [ ]:
# stop
auto_drone.hover()

In [ ]:
# rotate 90 degree to the right
auto_drone.rotate(90)

In [ ]:
# move higher 1 m
auto_drone.move_vertically(1) 

In [ ]:
# camera moves up 45 degree
auto_drone.change_camera_pitch(-45)

In [ ]:
f.write('done intervention'.format(i, cur_yaw_degree, cur_z))
f.close()

### End intervention